<a href="https://colab.research.google.com/github/SHEIKHMOHAMMEDUMAR/Mini-Project/blob/main/DS_MINOR_MAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Autotime to show the time taken by each cell to execute.

!pip install ipython-autotime
%load_ext autotime

In [ ]:
# Libabries required.

import os
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.metrics import accuracy_score, confusion_matrix
import pickle

In [ ]:
#Data Collection and Preprocessing;

target_name = []
flat_data = []
images = []

DD= '/content/drive/MyDrive/Images'
category = ['apple','ferrari','airplane']

for c in category:
  class_num = category.index(c)
  path = os.path.join(DD,c)
  for img in os.listdir(path):
    img_a = imread(os.path.join(path,img))
    img_r = resize(img_a,(150,150,3))
    flat_data.append(img_r.flatten())
    images.append(img_r)
    target_name.append(class_num)

flat_data = np.array(flat_data)
target_name = np.array(target_name)
images = np.array(images)

In [ ]:
#Spliting Dataset;
 
x_train, x_test, y_train, y_test = train_test_split(flat_data, target_name,
                                                    test_size=0.3, random_state=100)

In [ ]:
#Model buliding

p_g = [
       {'C':[1,10,100,1000],'kernel':['linear']},
       {'C':[1,10,100,1000],'gamma':[0.001,0.0001],'kernel':['rbf']},
] 
svc = svm.SVC(probability = True)
clf = GridSearchCV(svc, p_g)
clf.fit(x_train, y_train)

In [ ]:
# Ouput Prediction 

y_p = clf.predict(x_test)
y_p

In [ ]:
y_test

In [ ]:
# Accuracy of the Classification Model.

accuracy_score(y_p, y_test)

In [ ]:
# Confusion matrix.

confusion_matrix(y_p, y_test)

In [ ]:
# Saving the Builted Model.

pickle.dump(clf,open('img_model.p','wb'))

In [ ]:
# loading the model.

model = pickle.load(open('img_model.p','rb'))

In [ ]:
# Classifing the purely unseen data from google.

flat_data = []
url = input('Enter your URL: ')
img = imread(url)
img_re = resize(img,(150,150,3))
flat_data.append(img_re.flatten())
flat_data = np.array(flat_data)
print(img.shape)
plt.imshow(img_re)
y_out = model.predict(flat_data)
y_out = category[y_out[0]]
print(f'Predicted output: {y_out}')

In [ ]:
#Installing Streamlit and Pyngrok for Deployment.

!pip install streamlit
!pip install pyngrok==4.1.1
from pyngrok import ngrok

In [ ]:
#Writing the app file to the streamlit server.

%%writefile app.py
import streamlit as st
import numpy as np
from skimage.io import imread
from skimage.transform import resize
import pickle
from PIL import Image
st.title('Image Classifier using Machine Learning')
st.text('Upload a Image')
model = pickle.load(open('img_model.p','rb'))
u_f = st.file_uploader('Choose an Image...', type='jpg')
if u_f is not None:
  img = Image.open(u_f)
  st.image(img, caption='Uploaded Image')

  if st.button('PREDICT'):
    category = ['apple','ferrari','airplane'] 
    st.write('Result...')
    flat_data=[]
    img = np.array(img)
    img_re = resize(img,(150,150,3))
    flat_data.append(img_re.flatten())
    flat_data = np.array(flat_data)
    y_out = model.predict(flat_data)
    y_out = category[y_out[0]]
    st.title(f'Predicted output: {y_out}')

In [ ]:
# Connecting to the server port number 80 using ngrok.

!nohup streamlit run app.py &
url = ngrok.connect(port='8501')
url
